### Setup & Definition

In [ ]:
import torch
import random
import torch.optim as optim
import torch.nn as nn
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#deallocate all cuda memory
torch.cuda.empty_cache()

#print cuda memory
print(torch.cuda.memory_allocated())

In [2]:
class AUNNModel(nn.Module):
    def __init__(
        self, 
        embedding_dim:int, 
        output_dim:int, 
        num_layers:int, 
        hidden_dim:int,
        context_len:int=10_000):        

        assert num_layers >= 2, "Number of layers must be at least 2"

        super(AUNNModel, self).__init__() 
    
        self.embedding_dim = embedding_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

        # Initialize embedding parameters
        thresh = 0.1
        alpha = -np.log(thresh) / (context_len ** 2)
        M = embedding_dim // 2
        self.w = torch.tensor(np.random.normal(0, np.sqrt(2 * alpha), M), device=device, dtype=torch.float32)
        self.b = torch.tensor(np.random.uniform(0, 2 * np.pi, M), device=device, dtype=torch.float32)

        # Input Layer
        self.input_layer =  nn.Linear(self.embedding_dim, self.hidden_dim)

        # Hidden Layers
        self.layers = nn.ModuleList()
        for _ in range(self.num_layers - 2):  # Exclude input and output layers
            self.layers.append(nn.Sequential(
                nn.Linear(self.hidden_dim, self.hidden_dim),
                nn.SiLU()
            ))

        # Output Layer
        self.output_layer = nn.Linear(self.hidden_dim, self.output_dim)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):

        for m in self.modules():
            if isinstance(m, nn.Linear):
                # Kaiming He initialization for Swish activation
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def count_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def encode(self, x: torch.Tensor):

        dim = self.embedding_dim
        inps = torch.outer(x, self.w) + self.b
        embed = np.sqrt(2 / dim) * torch.concatenate([
            torch.cos(inps),
            torch.sin(inps)
        ], dim=-1)
        return embed

    def forward(self, indices):
        
        x = self.encode(indices)
        x = self.input_layer(x)
        x = x + nn.SiLU()(x)

        for layer in self.layers:
            x = x + layer(x)  # MLP output with skip connection

        x = self.output_layer(x)

        return x

In [67]:
# Define a function to save the model checkpoint
def save_checkpoint(model, params, optimizer, losses, filename="checkpoint_rff.pth"):
    
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'losses': losses,
    }

    keys = ['embedding_dim', 'output_dim', 'num_layers', 'hidden_dim', 'context_len']
    assert all(k in params for k in keys)
    for k in keys:
        checkpoint[k] = params[k]

    torch.save(checkpoint, filename)
    print(f"Checkpoint saved with loss {losses[-1]:.4f}")

In [68]:
def load_checkpoint(filename="checkpoint_rff.pth"):

    checkpoint = torch.load(filename, weights_only=True)
    
    keys = ['embedding_dim', 'output_dim', 'num_layers', 'hidden_dim', 'context_len']
    params = {k: checkpoint[k] for k in keys}
    
    model = AUNNModel(**params)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    
    optimizer = optim.AdamW(model.parameters())
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    losses = checkpoint['losses']

    print(f"Checkpoint loaded: loss {losses[-1]:.4f}")

    return model, optimizer, losses

### Train

In [44]:
# Hyperparameters

embedd_dim = 4096
num_layers = 4     # Must be even and at least 2 (bc of skip connections)
hidden_dim = 128   # Size of hidden layers
batch_size = 8192  # Adjust batch size for efficiency
num_epochs = 2000
context_len = 8

In [ ]:
#really long repeated pattern
'''
text = "abc" * 10_000  # Repeat the sequence to create a long string
'''

#really long random pattern
options = ['|aaa','|bbb','|ccc']
num_sequences = 15_000
num_repeats = num_sequences * len(options)
num_repeats = num_repeats - num_repeats % batch_size # ensure num_repeats is a multiple of batch_size
options = options * num_repeats
random.seed(42)
random.shuffle(options)
text = ''.join(options)
print(len(text))

In [40]:
vocab = sorted(set(text))
token_to_id = {token: id for id, token in enumerate(vocab)}
id_to_token = {id: token for token, id in token_to_id.items()}
token_ids = [token_to_id[char] for char in text]
token_ids = torch.tensor(token_ids, dtype=torch.long).to(device)
indices = torch.arange(len(token_ids)).to(device)

In [43]:
# Initialize the model, loss function, and optimizer
model = AUNNModel(embedd_dim, len(vocab), num_layers, hidden_dim, context_len=context_len).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters())
losses = []
accuracies = []

In [ ]:
# Training Loop
total_steps = len(text) // batch_size
if len(text) % batch_size != 0:
    total_steps += 1

for epoch in tqdm(range(num_epochs)):
    model.train()
    start_idxs = list(range(0, len(text), batch_size))
    # random.seed(epoch)
    # random.shuffle(start_idxs)
    for start_idx in start_idxs:
        end_idx = start_idx + batch_size
        
        inputs = indices[start_idx:end_idx]
        targets = token_ids[start_idx:end_idx]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        cur_loss = loss.item()
        losses.append(cur_loss)

        #calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == targets).sum().item()
        cur_acc = correct / batch_size
        accuracies.append(cur_acc)
        
    avg_loss = sum(losses[-len(start_idxs):]) / len(start_idxs)
    avg_acc = sum(accuracies[-len(start_idxs):]) / len(start_idxs)
    print(f"Epoch [{epoch+1}/{num_epochs}] completed, Average Loss: {avg_loss:.6f}, Average Accuracy: {avg_acc:.4f}")

In [ ]:
steps_per_epoch = len(text) // batch_size
print(steps_per_epoch)

In [ ]:
params = {
    'embedding_dim': embedd_dim,
    'output_dim': len(vocab),
    'num_layers': num_layers,
    'hidden_dim': hidden_dim
}
save_checkpoint(model, params, optimizer, losses, filename="sequence/checkpoint_rff.pth")

In [ ]:
model, optimizer, losses = load_checkpoint(filename="sequence/checkpoint_rff.pth")

In [ ]:
# Plot the loss curves
plt.figure(figsize=(40, 6))
plt.plot(losses[::], label="Training Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.legend()
plt.show()

# Plot the accuracy curves
plt.figure(figsize=(40, 6))
plt.plot(accuracies[::], label="Training Accuracy")
plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Accuracy")
plt.legend()
plt.show()

In [ ]:
# Generate Text Function
def generate_text(model, start_index, length):
    model.eval()
    generated_tokens = []
    indices = torch.arange(start_index, start_index + length).to(device)
    with torch.no_grad():
        outputs = model(indices)
        _, predicted = torch.max(outputs, 1)
    for id in predicted:
        token = id_to_token.get(id.item(), "<UNK>")
        generated_tokens.append(token)
    return ''.join(generated_tokens)

# Generate text starting from the next index after the training data
start_index = 0
generated_text = generate_text(model, start_index=start_index, length=1000)
print("Generated Text:")
print(generated_text)

# Generate text starting from the next index after the training data
start_index = len(text)
generated_text = generate_text(model, start_index=start_index, length=100)
print("Generated Text:")
print(generated_text)

In [ ]:
N = len(text) - 1
preds = generate_text(model, start_index=N+1, length=50_000)
count = {}
for char in preds:
    count[char] = count.get(char, 0) + 1

plt.bar(count.keys(), count.values())
plt.show()

In [ ]:
# Evaluate the accuracy of the model on the dataset via random sample of 100k characters
import random

# Number of samples
num_samples = 100_000

# Randomly select indices from the dataset
sample_indices = random.sample(range(len(text)), num_samples)

# Get positions and targets for the sampled indices
inputs = []
outputs = []
for idx in sample_indices:
    inp = embeds[idx]
    inputs.append(inp)
    output = token_ids[idx]
    outputs.append(output)

# Concatenate the inputs and outputs
inputs = torch.stack(inputs).to(device)
outputs = torch.stack(outputs).to(device)

# Get model predictions
model.eval()
with torch.no_grad():
    predictions = model(inputs)
    _, predicted = torch.max(predictions, 1)
    
# Calculate accuracy
correct = (predicted == outputs).sum().item()
accuracy = correct / num_samples
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Example usage
total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params}")

In [ ]:
N = len(text) - 1

#see value before conditioning
generated_text = generate_text(model, start_index=N+1, length=150)
print("Original Text:")
print(generated_text) # "|a..."
print("")

#conditioning the model
conditioning_targets = ['|','c']  # Desired tokens at N+1 and N+2, this also works if you use "|b"
conditioning_positions = [N+1, N+2]
# conditioning_targets = ['c','a','b'] #show conditioning works on discontinuous tokens
# conditioning_positions = [N+2, N+7, N+12]

conditioning_target_indices = [token_to_id[token] for token in conditioning_targets]
targets_tensor = torch.tensor(conditioning_target_indices, dtype=torch.long).to(device)
positions_tensor = torch.tensor(conditioning_positions)
inputs = positional_encoding(positions_tensor, embedd_dim).to(device)

criterion = nn.CrossEntropyLoss()
conditioning_optimizer = optim.SGD(model.parameters(), lr=1e-3)
model.train()
num_conditioning_steps = 10

for step in range(num_conditioning_steps):
    conditioning_optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets_tensor)
    loss.backward()
    conditioning_optimizer.step()
    print(f"Conditioning Step {step+1}/{num_conditioning_steps}, Loss: {loss.item():.6f}")

#see value after conditioning
generated_text = generate_text(model, start_index=N+1, length=150)
print("")
print("Text after conditioning:")
print(generated_text) # "|ccc|..." shows conditioning works because of new values for N+3 and N+4